In [7]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import sqlite3
from sklearn import preprocessing

In [8]:
pd.options.display.max_columns = None
players = pd.read_csv('players_mejor_3.csv')

In [9]:
players = players[players["best_of"] == 3]

In [10]:
#borramos las columnas correspondientes a variables que se encuentran desagregadas por mapa y las de resumen del partido total
players.drop(columns=['m1_kills', 'm1_assists', 'm1_deaths', 'm2_kills', 'm2_assists', 'm2_deaths', 'm3_kills', 'm3_assists', 'm3_deaths', 'kills', 'assists', 'deaths', 'hs', 'flash_assists', 'kast', 'adr', 'fkdiff', 'rating','kills_ct', 'deaths_ct', 'kddiff_ct', 'adr_ct','kast_ct','rating_ct','kills_t','deaths_t','kddiff_t','adr_t','kast_t','rating_t', 'kddiff','m1_kills_ct','m1_deaths_ct','m1_kddiff_ct','m1_adr_ct','m1_kast_ct','m1_rating_ct','m1_kills_t','m1_deaths_t','m1_kddiff_t','m1_adr_t','m1_kast_t','m1_rating_t','m2_kills_ct','m2_deaths_ct','m2_kddiff_ct','m2_adr_ct','m2_kast_ct','m2_rating_ct','m2_kills_t','m2_deaths_t','m2_kddiff_t','m2_adr_t','m2_kast_t','m2_rating_t','m3_kills_ct','m3_deaths_ct','m3_kddiff_ct','m3_adr_ct','m3_kast_ct','m3_rating_ct','m3_kills_t','m3_deaths_t','m3_kddiff_t','m3_kast_t','m3_rating_t','m3_adr_t'],axis=1,inplace=True)

In [11]:
players.sample(3)

,date,player_name,team,opponent,country,player_id,match_id,event_id,event_name,best_of,map_1,map_2,map_3,m1_hs,m1_flash_assists,m1_kast,m1_kddiff,m1_adr,m1_fkdiff,m1_rating,m2_hs,m2_flash_assists,m2_kast,m2_kddiff,m2_adr,m2_fkdiff,m2_rating,m3_hs,m3_flash_assists,m3_kast,m3_kddiff,m3_adr,m3_fkdiff,m3_rating
31327,2019-07-11,gargoyle,madlikewizards,Justice,Australia,14332,2334834,4741,ESL Australia & NZ Championship Season 9,3,Overpass,Mirage,NaN,9,0.0,69.0,-6,65.3,-2,0.84,12.0,1.0,75.0,10.0,108.7,-1.0,1.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN
170032,2016-08-14,HSK,MVP Project,autocracy,Korea,9174,2303905,2303,VSL Season 1,3,Nuke,Overpass,NaN,6,NaN,51.9,-12,65.1,-1,0.59,7.0,NaN,75.9,0.0,70.8,2.0,1.09,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106884,2018-01-26,Marek,EHOME,Signature,Taiwan,8598,2318725,3507,CSEsport.com Asia Invitational,3,Inferno,Train,Overpass,9,2.0,85.2,1,55.5,-3,1.02,10.0,1.0,73.3,-2.0,55.2,0.0,0.84,12.0,1.0,78.6,1.0,86.3,2.0,1.15


In [12]:
players.shape

(178728, 34)

In [13]:
# Crear una conexión a una base de datos en memoria
conn = sqlite3.connect(":memory:")

# Guardar los datos en una tabla temporal
players.to_sql("players", conn, if_exists="replace")

# Ejecutar la consulta SQL para filtrar los resultados
# al armar el dataset nuevo descartamos las columnas que son ids (menos match_id que lo usamos despues para transformar team y opponent), date, player_name y event_name al no utilizarlas
# tambien borramos las columnas correspondientes a valores futuros al momento de predecir por lo que no debemos mantener en entre los datos
query = """
SELECT match_id, team, opponent, country, map_1 AS first_map, m1_hs AS first_map_hs, m1_kast AS first_map_kast, m1_kddiff AS first_map_kddiff, m1_adr AS first_map_adr,
    m1_fkdiff AS first_map_fkdiff, m1_rating AS first_map_rating, map_2 AS second_map, m2_kddiff AS second_map_kddiff
FROM players;
"""

#aca lo que hacemos es para no perder los registros que solo se jugaron dos mapas porque los ganaron de corrido
#si el map3 tiene NaN no creamos el segundo registro
query2 = """
SELECT match_id, team, opponent, country, map_2 AS first_map, m2_hs AS first_map_hs, m2_kast AS first_map_kast, m2_kddiff AS first_map_kddiff, m2_adr AS first_map_adr,
    m2_fkdiff AS first_map_fkdiff, m2_rating AS first_map_rating, map_3 AS second_map, m3_kddiff AS second_map_kddiff
FROM players
WHERE map_3 IS NOT NULL;
"""

# Obtener un dataset con los registros de la primer query
players_nuevo_first = pd.read_sql_query(query, conn)
# Obterner un dataset con los registros de la segunda query
players_nuevo_second = pd.read_sql_query(query2, conn)

conn.close()

# Concatenamos ambos dataset para obtener el dataset final
players_nuevo = pd.concat([players_nuevo_first, players_nuevo_second], ignore_index=True)


In [14]:
players_nuevo.sample(3)

,match_id,team,opponent,country,first_map,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map,second_map_kddiff
70774,2325750,Heroic,fnatic,Denmark,Mirage,10.0,63.0,11.0,105.3,6.0,1.54,Inferno,-9.0
99648,2319024,MANS NOT HOT,YouCorn,Denmark,Inferno,4.0,94.7,18.0,97.1,2.0,1.88,Mirage,9.0
244358,2298526,Reason,ESC,Denmark,Overpass,18.0,NaN,4.0,NaN,3.0,1.19,Dust2,-7.0


In [15]:
players_nuevo.shape

(244370, 13)

In [16]:
num_country = players_nuevo['country'].nunique()
num_first_map = players_nuevo['first_map'].nunique()
num_second_map = players_nuevo['second_map'].nunique()

# Imprimir la cantidad de valores distintos
print("Cantidad de valores distintos en la columna 'country':", num_country)
print("Cantidad de valores distintos en la columna 'first_map':", num_first_map)
print("Cantidad de valores distintos en la columna 'second_map':", num_second_map)

Cantidad de valores distintos en la columna 'country': 109
Cantidad de valores distintos en la columna 'first_map': 10
Cantidad de valores distintos en la columna 'second_map': 10


In [17]:
# hacer one hot enconder a country, first_map y second_map:
# creamos las columnas encodeadas y las formateamos
players_nuevo = pd.get_dummies(players_nuevo, columns=['country','first_map', 'second_map'])

players_nuevo.columns = players_nuevo.columns.str.lower()
BETTER_COLUMN_NAMES = {
    'country_bosnia and herzegovina': 'country_bosnia_and_herzegovina',
    'country_costa rica': 'country_costa_rica',
    'country_czech republic': 'country_czech_republic',
    'country_dominican republic': 'country_dominican_republic',
    'country_el salvador': 'country_el_salvador',
    'country_faroe islands': 'country_faroe_islands',
    'country_hong kong': 'country_hong_kong',
    'country_new zealand': 'country_new_zealand',
    'country_saudi arabia': 'country_saudi_arabia',
    'country_south africa': 'country_south_africa',
    'country_sri lanka': 'country_sri_lanka',
    'country_united arab emirates': 'country_united_arab_emirates',
    'country_united kingdom': 'country_united_kingdom',
    'country_united states': 'country_united_states'
}
players_nuevo.rename(columns=BETTER_COLUMN_NAMES, inplace=True)


In [18]:
players_nuevo.shape

(244370, 139)

In [19]:
# codificacion de variables team y opponent como el puesto que ocupaba cada equipo en el ranking de cs oficial en el dia del partido
# para esto utilizamos el dataset results.csv del mismo lugar donde descargamos el dataset que estamos utilizando
# lo que hacemos es por cada match_id, reemplazar las columnas team y opponent por el numero de ranking (columnas team_ranking y opponent_ranking)

In [20]:
# este dataset contiene tres registros por cada partido (el resultado de cada mapa) por lo que lo leemos y nos quedamos con cualquier de esos tres registros, ya que solo nos interesa el ranking y son los mismos valores en los tres
results = pd.read_csv('results.csv').drop_duplicates(subset='match_id', keep='first')

# vemos un ejemplo de un registro de results
results.sample(1)

,date,team_1,team_2,_map,result_1,result_2,map_winner,starting_ct,ct_1,t_2,t_1,ct_2,event_id,match_id,rank_1,rank_2,map_wins_1,map_wins_2,match_winner
33357,2017-06-30,Mask Off,Eanix,Cobblestone,16,10,1,2,9,2,7,8,2880,2312117,62,78,16,10,1


In [21]:
# creamos las columnas nuevas que van a alojar el valor de ranking
players_nuevo["team_ranking"] = -1
players_nuevo["opponent_ranking"] = -1

#recorremos players_nuevo
for index, registro in players_nuevo.iterrows():
    #buscamos el registro de results que coincida con el actual match_id del registro de players_nuevo
    row = results.loc[results['match_id'] == registro['match_id']]
    #si encontramos el match en results
    if len(row) > 0:
        #asignamos las nuevas columnas del ranking de cada equipo dependiendo como venian ordenadas
        if registro['team'] == row['team_1'].values[0] and registro['opponent'] == row['team_2'].values[0]:
            players_nuevo.loc[index, 'team_ranking'] = row['rank_1'].values[0]
            players_nuevo.loc[index, 'opponent_ranking'] = row['rank_2'].values[0]
        elif registro['opponent'] == row['team_1'].values[0] and registro['team'] == row['team_2'].values[0]:
            players_nuevo.loc[index, 'team_ranking'] = row['rank_2'].values[0]
            players_nuevo.loc[index, 'opponent_ranking'] = row['rank_1'].values[0]

players_nuevo.sample(3)

,match_id,team,opponent,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map_kddiff,country_albania,country_algeria,country_argentina,country_armenia,country_australia,country_austria,country_azerbaijan,country_bahrain,country_bangladesh,country_belarus,country_belgium,country_bolivia,country_bosnia_and_herzegovina,country_brazil,country_brunei,country_bulgaria,country_cambodia,country_canada,country_chile,country_china,country_colombia,country_costa_rica,country_croatia,country_cyprus,country_czech_republic,country_denmark,country_dominican_republic,country_ecuador,country_egypt,country_el_salvador,country_estonia,country_faroe_islands,country_finland,country_france,country_georgia,country_germany,country_greece,country_guatemala,country_hong_kong,country_hungary,country_iceland,country_india,country_indonesia,country_iran,country_iraq,country_ireland,country_israel,country_italy,country_japan,country_jordan,country_kazakhstan,country_korea,country_kuwait,country_kyrgyzstan,country_laos,country_latvia,country_lebanon,country_libya,country_lithuania,country_luxembourg,country_macau,country_macedonia,country_malaysia,country_malta,country_mexico,country_moldova,country_mongolia,country_montenegro,country_morocco,country_myanmar,country_nepal,country_netherlands,country_new_zealand,country_norway,country_pakistan,country_palestine,country_panama,country_paraguay,country_peru,country_philippines,country_poland,country_portugal,country_qatar,country_romania,country_russia,country_saudi_arabia,country_serbia,country_singapore,country_slovakia,country_slovenia,country_south_africa,country_spain,country_sri_lanka,country_sweden,country_switzerland,country_syria,country_taiwan,country_thailand,country_tunisia,country_turkey,country_turkmenistan,country_ukraine,country_united_arab_emirates,country_united_kingdom,country_united_states,country_uruguay,country_uzbekistan,country_venezuela,country_vietnam,first_map_cache,first_map_cobblestone,first_map_default,first_map_dust2,first_map_inferno,first_map_mirage,first_map_nuke,first_map_overpass,first_map_train,first_map_vertigo,second_map_cache,second_map_cobblestone,second_map_default,second_map_dust2,second_map_inferno,second_map_mirage,second_map_nuke,second_map_overpass,second_map_train,second_map_vertigo,team_ranking,opponent_ranking
224087,2313175,GoodJob,Squared,10.0,65.2,-5.0,74.3,1.0,0.83,4.0,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,-1,-1
218860,2317070,Nexus,forZe,13.0,84.6,15.0,101.6,5.0,1.71,-6.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,F

In [22]:
#borramos los registros de players_nuevo que quedan con columnas nuevas ya que no tenian ocurrencia en results, perdiendo datos
indices_a_eliminar = []
for index, registro in players_nuevo.iterrows():
    # Verifica si las columnas seleccionadas contienen el valor -1
    if registro['team_ranking'] == -1 or registro['opponent_ranking'] == -1:
        # guardamos el indice para eliminarlo despues
        indices_a_eliminar.append(index)

# Elimina los registros en un solo paso
players_nuevo.drop(indices_a_eliminar, inplace=True)

#borramos tambien las columnas match_id, team y opponent que ya no utilizamos
players_nuevo.drop(columns=['team', 'opponent', 'match_id'],axis=1,inplace=True)
        
players_nuevo.sample(3)

,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map_kddiff,country_albania,country_algeria,country_argentina,country_armenia,country_australia,country_austria,country_azerbaijan,country_bahrain,country_bangladesh,country_belarus,country_belgium,country_bolivia,country_bosnia_and_herzegovina,country_brazil,country_brunei,country_bulgaria,country_cambodia,country_canada,country_chile,country_china,country_colombia,country_costa_rica,country_croatia,country_cyprus,country_czech_republic,country_denmark,country_dominican_republic,country_ecuador,country_egypt,country_el_salvador,country_estonia,country_faroe_islands,country_finland,country_france,country_georgia,country_germany,country_greece,country_guatemala,country_hong_kong,country_hungary,country_iceland,country_india,country_indonesia,country_iran,country_iraq,country_ireland,country_israel,country_italy,country_japan,country_jordan,country_kazakhstan,country_korea,country_kuwait,country_kyrgyzstan,country_laos,country_latvia,country_lebanon,country_libya,country_lithuania,country_luxembourg,country_macau,country_macedonia,country_malaysia,country_malta,country_mexico,country_moldova,country_mongolia,country_montenegro,country_morocco,country_myanmar,country_nepal,country_netherlands,country_new_zealand,country_norway,country_pakistan,country_palestine,country_panama,country_paraguay,country_peru,country_philippines,country_poland,country_portugal,country_qatar,country_romania,country_russia,country_saudi_arabia,country_serbia,country_singapore,country_slovakia,country_slovenia,country_south_africa,country_spain,country_sri_lanka,country_sweden,country_switzerland,country_syria,country_taiwan,country_thailand,country_tunisia,country_turkey,country_turkmenistan,country_ukraine,country_united_arab_emirates,country_united_kingdom,country_united_states,country_uruguay,country_uzbekistan,country_venezuela,country_vietnam,first_map_cache,first_map_cobblestone,first_map_default,first_map_dust2,first_map_inferno,first_map_mirage,first_map_nuke,first_map_overpass,first_map_train,first_map_vertigo,second_map_cache,second_map_cobblestone,second_map_default,second_map_dust2,second_map_inferno,second_map_mirage,second_map_nuke,second_map_overpass,second_map_train,second_map_vertigo,team_ranking,opponent_ranking
128122,9.0,75.0,7.0,81.9,2.0,1.29,-1.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,32,23
109414,7.0,50.0,-6.0,55.4,-1.0,0.63,-11.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,43,16
5403,9.0,75.9,3.0,82.

In [23]:
players_nuevo.shape

(165946, 138)

In [24]:
# normalizamos todas las columnas necesarias
columnas_a_normalizar = ['first_map_hs','first_map_kast', 'first_map_kddiff', 'first_map_adr', 'first_map_fkdiff', 'team_ranking', 'opponent_ranking']

# Crear un objeto MinMaxScaler para normalizar las columnas
scaler = preprocessing.MinMaxScaler()

# Normalizar las columnas seleccionadas
players_nuevo[columnas_a_normalizar] = scaler.fit_transform(players_nuevo[columnas_a_normalizar])



In [25]:
#borramos los registros de players_nuevo que quedan con valores nulos en first_map_kddiff y second_map_kddiff (siguen teniendo -1 como al inicializar)
indices_a_eliminar = []
for index, registro in players_nuevo.iterrows():
    # Verifica si las columnas seleccionadas contienen el valor -1
    if registro.isnull().any():
        # guardamos el indice para eliminarlo despues
        indices_a_eliminar.append(index)

# Elimina los registros en un solo paso
players_nuevo.drop(indices_a_eliminar, inplace=True)

In [26]:
players_nuevo.sample(3)

,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map_kddiff,country_albania,country_algeria,country_argentina,country_armenia,country_australia,country_austria,country_azerbaijan,country_bahrain,country_bangladesh,country_belarus,country_belgium,country_bolivia,country_bosnia_and_herzegovina,country_brazil,country_brunei,country_bulgaria,country_cambodia,country_canada,country_chile,country_china,country_colombia,country_costa_rica,country_croatia,country_cyprus,country_czech_republic,country_denmark,country_dominican_republic,country_ecuador,country_egypt,country_el_salvador,country_estonia,country_faroe_islands,country_finland,country_france,country_georgia,country_germany,country_greece,country_guatemala,country_hong_kong,country_hungary,country_iceland,country_india,country_indonesia,country_iran,country_iraq,country_ireland,country_israel,country_italy,country_japan,country_jordan,country_kazakhstan,country_korea,country_kuwait,country_kyrgyzstan,country_laos,country_latvia,country_lebanon,country_libya,country_lithuania,country_luxembourg,country_macau,country_macedonia,country_malaysia,country_malta,country_mexico,country_moldova,country_mongolia,country_montenegro,country_morocco,country_myanmar,country_nepal,country_netherlands,country_new_zealand,country_norway,country_pakistan,country_palestine,country_panama,country_paraguay,country_peru,country_philippines,country_poland,country_portugal,country_qatar,country_romania,country_russia,country_saudi_arabia,country_serbia,country_singapore,country_slovakia,country_slovenia,country_south_africa,country_spain,country_sri_lanka,country_sweden,country_switzerland,country_syria,country_taiwan,country_thailand,country_tunisia,country_turkey,country_turkmenistan,country_ukraine,country_united_arab_emirates,country_united_kingdom,country_united_states,country_uruguay,country_uzbekistan,country_venezuela,country_vietnam,first_map_cache,first_map_cobblestone,first_map_default,first_map_dust2,first_map_inferno,first_map_mirage,first_map_nuke,first_map_overpass,first_map_train,first_map_vertigo,second_map_cache,second_map_cobblestone,second_map_default,second_map_dust2,second_map_inferno,second_map_mirage,second_map_nuke,second_map_overpass,second_map_train,second_map_vertigo,team_ranking,opponent_ranking
214886,0.184211,0.685039,0.466667,0.460256,0.56,1.12,3.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,0.096859,0.196335
118560,0.315789,0.625422,0.533333,0.512507,0.36,1.22,-4.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,Fals

In [27]:
# por ultimo, convertimos second_map_kddiff en las etiquetas 1 o 0 dependiendo si son positivas o negativas, respectivamente
players_nuevo['second_map_kddiff'] = np.where(players_nuevo['second_map_kddiff'] >= 0, 1, 0)

players_nuevo.sample(3)

,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map_kddiff,country_albania,country_algeria,country_argentina,country_armenia,country_australia,country_austria,country_azerbaijan,country_bahrain,country_bangladesh,country_belarus,country_belgium,country_bolivia,country_bosnia_and_herzegovina,country_brazil,country_brunei,country_bulgaria,country_cambodia,country_canada,country_chile,country_china,country_colombia,country_costa_rica,country_croatia,country_cyprus,country_czech_republic,country_denmark,country_dominican_republic,country_ecuador,country_egypt,country_el_salvador,country_estonia,country_faroe_islands,country_finland,country_france,country_georgia,country_germany,country_greece,country_guatemala,country_hong_kong,country_hungary,country_iceland,country_india,country_indonesia,country_iran,country_iraq,country_ireland,country_israel,country_italy,country_japan,country_jordan,country_kazakhstan,country_korea,country_kuwait,country_kyrgyzstan,country_laos,country_latvia,country_lebanon,country_libya,country_lithuania,country_luxembourg,country_macau,country_macedonia,country_malaysia,country_malta,country_mexico,country_moldova,country_mongolia,country_montenegro,country_morocco,country_myanmar,country_nepal,country_netherlands,country_new_zealand,country_norway,country_pakistan,country_palestine,country_panama,country_paraguay,country_peru,country_philippines,country_poland,country_portugal,country_qatar,country_romania,country_russia,country_saudi_arabia,country_serbia,country_singapore,country_slovakia,country_slovenia,country_south_africa,country_spain,country_sri_lanka,country_sweden,country_switzerland,country_syria,country_taiwan,country_thailand,country_tunisia,country_turkey,country_turkmenistan,country_ukraine,country_united_arab_emirates,country_united_kingdom,country_united_states,country_uruguay,country_uzbekistan,country_venezuela,country_vietnam,first_map_cache,first_map_cobblestone,first_map_default,first_map_dust2,first_map_inferno,first_map_mirage,first_map_nuke,first_map_overpass,first_map_train,first_map_vertigo,second_map_cache,second_map_cobblestone,second_map_default,second_map_dust2,second_map_inferno,second_map_mirage,second_map_nuke,second_map_overpass,second_map_train,second_map_vertigo,team_ranking,opponent_ranking
17205,0.105263,0.662542,0.283333,0.298499,0.44,0.65,0,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,0.206806,0.835079
39270,0.052632,0.505062,0.250000,0.205114,0.40,0.49,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,

In [28]:
#chequeamos valores nulos
players_nuevo.isnull().sum()

first_map_hs           0
first_map_kast         0
first_map_kddiff       0
first_map_adr          0
first_map_fkdiff       0
                      ..
second_map_overpass    0
second_map_train       0
second_map_vertigo     0
team_ranking           0
opponent_ranking       0
Length: 138, dtype: int64

In [30]:
#HASTA ACA TENEMOS LISTO EL PREPROCESAMIENTO

#players_nuevo.to_csv('players_preprocesado.csv', index=False)
players_nuevo.sample(3)

,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map_kddiff,country_albania,country_algeria,country_argentina,country_armenia,country_australia,country_austria,country_azerbaijan,country_bahrain,country_bangladesh,country_belarus,country_belgium,country_bolivia,country_bosnia_and_herzegovina,country_brazil,country_brunei,country_bulgaria,country_cambodia,country_canada,country_chile,country_china,country_colombia,country_costa_rica,country_croatia,country_cyprus,country_czech_republic,country_denmark,country_dominican_republic,country_ecuador,country_egypt,country_el_salvador,country_estonia,country_faroe_islands,country_finland,country_france,country_georgia,country_germany,country_greece,country_guatemala,country_hong_kong,country_hungary,country_iceland,country_india,country_indonesia,country_iran,country_iraq,country_ireland,country_israel,country_italy,country_japan,country_jordan,country_kazakhstan,country_korea,country_kuwait,country_kyrgyzstan,country_laos,country_latvia,country_lebanon,country_libya,country_lithuania,country_luxembourg,country_macau,country_macedonia,country_malaysia,country_malta,country_mexico,country_moldova,country_mongolia,country_montenegro,country_morocco,country_myanmar,country_nepal,country_netherlands,country_new_zealand,country_norway,country_pakistan,country_palestine,country_panama,country_paraguay,country_peru,country_philippines,country_poland,country_portugal,country_qatar,country_romania,country_russia,country_saudi_arabia,country_serbia,country_singapore,country_slovakia,country_slovenia,country_south_africa,country_spain,country_sri_lanka,country_sweden,country_switzerland,country_syria,country_taiwan,country_thailand,country_tunisia,country_turkey,country_turkmenistan,country_ukraine,country_united_arab_emirates,country_united_kingdom,country_united_states,country_uruguay,country_uzbekistan,country_venezuela,country_vietnam,first_map_cache,first_map_cobblestone,first_map_default,first_map_dust2,first_map_inferno,first_map_mirage,first_map_nuke,first_map_overpass,first_map_train,first_map_vertigo,second_map_cache,second_map_cobblestone,second_map_default,second_map_dust2,second_map_inferno,second_map_mirage,second_map_nuke,second_map_overpass,second_map_train,second_map_vertigo,team_ranking,opponent_ranking
109885,0.184211,0.625422,0.45,0.455809,0.56,1.12,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,0.225131,0.044503
72398,0.184211,0.678290,0.55,0.545859,0.56,1.40,1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,F

In [31]:
#SI QUEREMOS SIN COUNTRIES
prefix = 'country_'
columns_to_drop = [col for col in players_nuevo.columns if col.startswith(prefix)]
players_nuevo_sin_paises = players_nuevo.drop(columns=columns_to_drop)
players_nuevo_sin_paises.to_csv('players_preprocesado_sin_paises.csv', index=False)
players_nuevo_sin_paises.sample(3)

,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map_kddiff,first_map_cache,first_map_cobblestone,first_map_default,first_map_dust2,first_map_inferno,first_map_mirage,first_map_nuke,first_map_overpass,first_map_train,first_map_vertigo,second_map_cache,second_map_cobblestone,second_map_default,second_map_dust2,second_map_inferno,second_map_mirage,second_map_nuke,second_map_overpass,second_map_train,second_map_vertigo,team_ranking,opponent_ranking
24828,0.026316,0.766029,0.500000,0.333519,0.56,1.19,1,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,0.013089,0.068063
93826,0.157895,0.759280,0.383333,0.419122,0.44,1.05,1,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,0.070681,0.240838
103498,0.131579,0.640045,0.283333,0.347415,0.48,0.82,0,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,0.201571,0.308901


## Definición de métrica:

Elegimos ACCURACY (precisión).
La precisión es la proporción de predicciones correctas realizadas por el modelo. Se calcula dividiendo el número de predicciones correctas entre el número total de predicciones.
Elegimos esta métrica ya que es especialmente util en clasificación y además es fácil de interpretar.
Su desventaja principal se encuentra en problemas cuyas clases en el conjunto de datos están desbalanceadas, cosa que no presentará inconvenientes en nuestro dataset (valor obtenido en tp1 -> kddiff positivo = 50,07% / kddiff negativo = 49,03%)

## División del dataset:

In [4]:
# Dividir el conjunto de datos en variable de entrada (entradas) y variable objetivo (salida)

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
# 60% train, 20% test, 20% validation
train, not_train = train_test_split(players_nuevo, test_size=0.6, random_state=42)
validation, test = train_test_split(not_train, test_size=0.5, random_state=42)

train_y = train['second_map_kddiff']
train_x = train.drop(columns=['second_map_kddiff'])

validation_y = validation['second_map_kddiff']
validation_x = validation.drop(columns=['second_map_kddiff'])

test_y = test['second_map_kddiff']
test_x = test.drop(columns=['second_map_kddiff'])

# Mostramos las dimensiones finales de cada porcion del dataset
train_x.shape, train_y.shape, validation_x.shape, validation_y.shape, test_x.shape, test_y.shape




NameError: name 'train_test_split' is not defined

## Feature engineering del dataset

In [209]:
# funcion para evaluar modelos
from sklearn import metrics

def evaluate_model(model, set_names=('train', 'validation'), title='', show_cm=False):
    if title:
        display(title)
        
    final_metrics = {
        'Accuracy': [],
        'Precision': [],
        'Recall': [],
        'F1': [],        
    }
        
    for i, set_name in enumerate(set_names):
        assert set_name in ['train', 'validation', 'test']
        set_data = globals()[set_name]  # <- hack feo...
        
        #ponemos nuestra variable a predecir
        y = set_data.second_map_kddiff
        y_pred = model.predict(set_data)
        
        final_metrics['Accuracy'].append(metrics.accuracy_score(y, y_pred))
        final_metrics['Precision'].append(metrics.precision_score(y, y_pred))
        final_metrics['Recall'].append(metrics.recall_score(y, y_pred))
        final_metrics['F1'].append(metrics.f1_score(y, y_pred))
        
        if show_cm:
            cm = metrics.confusion_matrix(y, y_pred)
            cm_plot = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, 
                                                     display_labels=['kddiff >= 0', 'kddiff < 0'])
            cm_plot.plot(cmap="Blues")
        
    display(pd.DataFrame(final_metrics, index=set_names))

## Entrenamiento de modelos:

### Regresion logística

In [269]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

regression_model = LogisticRegression(random_state=42, max_iter=1000)

regression_model.fit(train_x, train_y)

y_pred = regression_model.predict(test_x)

print("Accuracy on test: " + str(metrics.accuracy_score(test_y, y_pred)))

#evaluate_model(regression_model, title='Logistic Regression')

Accuracy on train: 0.5831791284543587


### Random forest

In [248]:
from sklearn.ensemble import RandomForestClassifier

forest_model = RandomForestClassifier()

forest_model.fit(train_x, train_y)

y_pred = forest_model.predict(test_x)

print("Accuracy on test: " + str(metrics.accuracy_score(test_y, y_pred)))

#evaluate_model(forest_model, title='Random Forest')

0.5755739471704252

### K nearest neighbors (KNN)

In [267]:
from sklearn.neighbors import KNeighborsClassifier

#Parametros varios:
K = 5

kkn_model = KNeighborsClassifier(n_neighbors=K)

kkn_model.fit(train_x, train_y)

y_pred = kkn_model.predict(test_x)

print("Accuracy on test: " + str(metrics.accuracy_score(test_y, y_pred)))

#evaluate_model(kkn_model, title='K nearest neighbors (KNN)')

Accuracy on train: 0.5363076234825224


### Dummie Classifier

In [270]:
from sklearn.dummy import DummyClassifier

# Crear el clasificador dummy
dummy_clf = DummyClassifier(strategy='uniform')

# Ajustar el clasificador dummy a tus datos
dummy_clf.fit(train_x, train_y)

# Realizar predicciones con el clasificador dummy
y_pred = dummy_clf.predict(test_x)


print("Accuracy on test: " + str(metrics.accuracy_score(test_y, y_pred)))

Accuracy on train: 0.5011895804949468


## Definición de hiperparametros implementando GridSearch: